문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템 

문제 설명

사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.

요구사항

1단계 체인: 장르를 입력받아 영화 1편 추천
2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
ChatPromptTemplate 사용 
: “system”  과 “human” 메시지를 지정합니다.
두 체인을 LCEL로 연결
각 단계의 결과를 모두 출력하여 과정 확인
구현 조건

입력: 영화 장르 (예: "액션", "로맨스", "공포")
1단계 출력: 추천 영화 제목과 간단한 설명
2단계 출력: 영화 줄거리 3줄 요약

예상 실행 결과
입력: "액션"
1단계 결과: "미션 임파서블" 추천
2단계 결과: 
영화: 미션 임파서블
줄거리:
1. 특수 요원 이든 헌트가 불가능한 임무를 수행합니다
2. 배신과 음모 속에서 진실을 찾아 나갑니다
3. 액션과 스릴러가 가득한 스파이 영화입니다


In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x00000184EA264440> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000184EA264EC0> root_client=<openai.OpenAI object at 0x00000184E9DE9550> root_async_client=<openai.AsyncOpenAI object at 0x00000184EA264C20> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
# chain 연결 (LCEL) prompt + llm + outputparser

prompt1 = ChatPromptTemplate.from_messages([
    ("system", "너는 영화 추천 전문가야."),
    ("human", "{genre} 장르 중 추천할만한 영화를 하나의 제목을 먼저 얘기하고 한 줄 설명을 줘."),
])


chain1 = prompt1 | llm | StrOutputParser()
print(type(chain1))

# 스트리밍 출력을 위한 요청
try:
    answer = chain1.stream({"genre": "액션"})

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)

except Exception as e:
    print(f"오류 발생: {e}")


<class 'langchain_core.runnables.base.RunnableSequence'>
'존 윅' - 전설적인 킬러 존 윅이 우연히 범죄 조직 보스의 개를 죽이면서 벌어지는 이야기입니다.

In [35]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "너는 영화 줄거리 요약가야."),
    ("human", "영화 {movie} 의 제목을 먼저 말하고 줄거리를 3줄로 요약해줘. ex ) 영화: [] \n 줄거리: []"),
])

In [36]:
try:
    chain2 = (
        {"movie": chain1}  # chain1의 출력을 movie 입력 변수로 전달
        | prompt2
        | llm
        | StrOutputParser()
    )
    response = chain2.invoke({"genre": "액션"})
    print(response)  
except Exception as e:
    print(f"오류 발생: {e}")

영화: 존 윅
줄거리: 전설적인 킬러 존 윅은 은퇴 후 평화로운 삶을 살고자 하지만, 우연히 범죄 조직의 표적이 되고 맙니다. 조직의 일원인 이오진에게 차량을 훔치기고, 그의 개마저 죽임을 당한 존 윅은 피할 수 없는 전쟁을 벌이게 됩니다. 존 윅은 전 세계의 범죄 조직과 싸우며 자신만의 복수를 시작합니다.
